In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets.rsna_breast_cancer import BreastCancerDataset, DatasetSplit
import backbones
from simplenet import SimpleNet
import utils
from torch.utils.data import DataLoader
import os

In [ ]:
img_dir = "../../Documents/rsna_breast_cancer"
csv_file = "train.csv"

ds = BreastCancerDataset(
    img_dir=img_dir,
    meta_data_csv_path=csv_file
)

val_ds = BreastCancerDataset(
    img_dir=img_dir,
    meta_data_csv_path=csv_file,
    split=DatasetSplit.VAL
)
train_loader = DataLoader(ds, batch_size=2, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=2, shuffle=True)

In [ ]:
backbone = backbones.load('efficientnet_b1')

device = 'cpu'

net = SimpleNet(device)
net.load(
        backbone=backbone,
        layers_to_extract_from=['conv_head'],
        device=device,
        input_shape=(3,224,224),
        pretrain_embed_dimension=1024,
        target_embed_dimension=1024,
        patchsize=3,
        embedding_size=1024,
        meta_epochs=1,
        aed_meta_epochs=1,
        gan_epochs=1,
        noise_std=0.05,
        dsc_layers=2,
        dsc_hidden=1024,
        dsc_margin=0.5,
        dsc_lr=0.0001,
        auto_noise=0,
        train_backbone=False,
        cos_lr=False,
        pre_proj=0,
        proj_layer_type=0,
        mix_noise=1,
    )
models_dir = "models"
dataset_name = "rsna_breast_cancer"
net.set_model_dir(models_dir, dataset_name)

In [ ]:
i_auroc, p_auroc, pro_auroc = net.train(train_loader, val_loader)
print(i_auroc, pro_auroc, pro_auroc)